#eXamine tutorial (with automation)

Trying to recreate the eXamine tutorial with automation ... 
Let's see how this goes

In [2]:
import py2cytoscape

# HTTP Client for Python
import requests
import json

# Standard JSON library
import json

# Basic Setup
PORT_NUMBER = 1234 #Yes, this is really the Cytoscape Basic Port

BASE = 'http://localhost:' + str(PORT_NUMBER) + '/v1/'


def executeRestCommand(namespace="",command="",args={}):
    postString = BASE + "commands/"+namespace+"/"+command
    firstarg = True;
    for arg in args:
        postString+=("?" if firstarg else "&")
        postString+=arg+"="+args[arg]
        if (firstarg):
            firstarg = False;
            
    print(postString)
    res = requests.get(postString)
    return res
    


In [ ]:
#First we import our demo network
res = executeRestCommand("network","import url",{"indexColumnSourceInteraction":"1","indexColumnTargetInteraction":"2","url":"https://raw.githubusercontent.com/ls-cwi/eXamine/master/data/edges.txt"})

print(res.text)


<p style="color:green;margin-left:10px;margin-top:0px;margin-bottom:5px;">DGraphView: suid=489641, model=edges.txt</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">Finished</p>



In [67]:
#TODO: Maybe retrieve the network SUID somehow for later reference, it is returned as DGraphView: suid = xyz
print(res.content.decode())

#res = cytoscape('network','import url',{"url":"https://raw.githubusercontent.com/ls-cwi/eXamine/master/data/edges.txt","sourceColumnIndex": "0"},verbose=True)


<p style="color:green;margin-left:10px;margin-top:0px;margin-bottom:5px;">DGraphView: suid=489641, model=edges.txt</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">Finished</p>



In [71]:
#Next we import node annotations
res = executeRestCommand(
    "table",
    "import url",
    {"firstRowAsColumnNames":"true",
     "keyColumnIndex" : "1",
     "startLoadRow" : "1",
     "dataTypeList":"s,s,f,f,f,s,s,s,sl,sl,sl,sl",
     "url":"https://raw.githubusercontent.com/ls-cwi/eXamine/master/data/nodes_induced.txt"})

print(res.text)

http://localhost:1234/v1/commands/table/import url?startLoadRow=1&firstRowAsColumnNames=true&url=https://raw.githubusercontent.com/ls-cwi/eXamine/master/data/nodes_induced.txt&keyColumnIndex=1&dataTypeList=s,s,f,f,f,s,s,s,sl,sl,sl,sl


<p style="color:green;margin-left:10px;margin-top:0px;margin-bottom:5px;">Mapped to tables:
   edges.txt root shared  node
</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">Finished</p>



In [3]:
#We select all nodes with the value "small" in the column "Module" TODO: Use previously fetched SUID

res = executeRestCommand(
    "network",
    "select",
    {"nodeList":"Module:small"})

http://localhost:1234/v1/commands/network/select?nodeList=Module:small


In [5]:
#As a next step we generate the groups via eXamine
res = executeRestCommand(
    "examine",
    "generate groups",
    {"selectedGroupColumns":"Process,Function,Component,Pathway"})

print(res.text)

http://localhost:1234/v1/commands/examine/generate groups?selectedGroupColumns=Process,Function,Component,Pathway


<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">Finished</p>



In [6]:
#Ok, time to enrich our newly greated group nodes with some interesting annotations
res = executeRestCommand(
    "table",
    "import url",
    {"firstRowAsColumnNames":"true",
     "keyColumnIndex" : "1",
     "startLoadRow" : "1",
     "url":"https://raw.githubusercontent.com/ls-cwi/eXamine/master/data/sets_induced.txt"})

print(res.text)

http://localhost:1234/v1/commands/table/import url?startLoadRow=1&firstRowAsColumnNames=true&keyColumnIndex=1&url=https://raw.githubusercontent.com/ls-cwi/eXamine/master/data/sets_induced.txt


<p style="color:green;margin-left:10px;margin-top:0px;margin-bottom:5px;">Mapped to tables:
   edges.txt root shared  node
</p>
<p style="color:black;margin-left:10px;margin-top:0px;margin-bottom:5px;">Finished</p>



In [ ]:
#Last Step: Either generate the eXamine window in Cytoscape or export the image
#TODO